## Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

## Drawing Hand Pose
<img src=https://google.github.io/mediapipe/images/mobile/hand_landmarks.png>

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

## Calculating Angles Between Joints

In [3]:
from matplotlib import pyplot as plt

In [4]:
joint_list = [
    [5,6,7],        # Index Finger
    [9,10,11],      # Middle Finger
    [13,14,15],     # Ring Finger
    [17,18,19]      # Pinky Finger
]

In [5]:
def draw_finger_angle(image, results, joint_list):

    # Loops through the different hands
    for hand in results.multi_hand_landmarks:

        # Loops through the sets of triplet joints
        for joint in joint_list:
            a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y, hand.landmark[joint[0]].z])    # Coordinates of 1st joint
            b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y, hand.landmark[joint[1]].z])    # Coordinates of 2nd joint
            c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y, hand.landmark[joint[2]].z])    # Coordinates of 3rd joint

            vector_1 = b - a
            vector_2 = b - c

            
            rad = np.arccos(
                (np.dot(vector_1, vector_2)/
                (np.sqrt(np.sum(np.square(vector_1)))*
                np.sqrt(np.sum(np.square(vector_2))))))

            
            deg = np.abs(rad * 180.0 / np.pi)

            if deg > 180:
                deg = 360 - deg

            cv2.putText(
                image, 
                str(round(deg, 3)), 
                tuple(np.multiply(b[:2], [640,480]).astype(int)), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, 
                (255,255,255), 
                2, 
                cv2.LINE_AA
                )

    return image         

## Draw Drumstick (Using finger angles)

In [6]:
drum_joint_list = [
    [0,17,19],
    [6,7,4]
]

In [7]:
def draw_drumstick(image, results, drum_joint_list):
    '''
    Output: POIs (2D-Array)

    Returns a 2D-array for left and right hand. 

    Each element of 2D-array contains 2 coordinates (idx 0: base of drumstick, idx 1: tip of drumstick)
    '''
    
    POIs = []

    # Loops through the different hands
    for hand in results.multi_hand_landmarks:
        POI = []
        # Loops through the sets of triplet joints
        for joint in drum_joint_list:
            a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y, hand.landmark[joint[0]].z])    # Coordinates of 1st joint
            b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y, hand.landmark[joint[1]].z])    # Coordinates of 2nd joint
            c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y, hand.landmark[joint[2]].z])    # Coordinates of 3rd joint

            joint_sim = np.add(a,b)/2
            joint_sim = np.add(joint_sim, c)/2
            joint_sim = tuple(np.multiply(joint_sim[:2], [640,480]).astype(int))

            POI.append(joint_sim)

        POI[1] =    (int((POI[1][0]-POI[0][0])*2.5 + POI[0][0]),
                    int((POI[1][1]-POI[0][1])*2.5 + POI[0][1]))
            
        POIs.append(POI)


    for poi in POIs:

        # poi[1]: tip of drumstick (type: tuple)
        cv2.circle(image, poi[1], 10, (255,255,255),5)

        # drawing stick itself
        cv2.line(image, poi[0], poi[1], (0,128,0), 9)

    return POIs


In [8]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        # Change colour format from BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Flip image along y axis
        image = cv2.flip(image, 1)

        # Set flag to false
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)

        # Set flag back to true
        image.flags.writeable = True

        # Change colour format from RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # print(results)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for idx, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)

            # draw_finger_angle(image, results, joint_list)

            # Draw drumsticks and get coordinates 
            POIs = draw_drumstick(image, results, drum_joint_list)
            print(POIs)


        cv2.imshow('Hand Tracking', image)

        #press q to quit
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

#run these 2 if ur cam crash (copy and run somewhere else)
cap.release()
cv2.destroyAllWindows()  

[[(261, 440), (196, 250)]]
[[(263, 439), (198, 246)]]
[[(265, 437), (200, 249)]]
[[(265, 431), (182, 258)]]
[[(263, 434), (178, 266)]]
[[(265, 436), (175, 268)]]
[[(259, 449), (174, 274)]]
[[(253, 449), (175, 276)]]
[[(250, 449), (172, 279)]]
[[(242, 455), (174, 282)]]
[[(240, 455), (170, 280)]]
[[(241, 454), (168, 284)]]
[[(242, 454), (167, 284)]]
[[(241, 455), (168, 285)]]
[[(241, 454), (168, 284)]]
[[(242, 455), (174, 280)]]
[[(251, 457), (171, 282)]]
[[(251, 457), (181, 284)]]
[[(251, 457), (178, 282)]]
[[(254, 456), (176, 283)]]
[[(254, 456), (176, 283)]]
[[(254, 455), (176, 285)]]
[[(253, 454), (178, 284)]]
[[(254, 453), (174, 285)]]
[[(254, 453), (176, 288)]]
[[(254, 454), (179, 286)]]
[[(254, 453), (181, 285)]]
[[(253, 453), (180, 288)]]
[[(252, 453), (182, 285)]]
[[(251, 453), (183, 285)]]
[[(251, 453), (181, 285)]]
[[(252, 453), (182, 283)]]
[[(251, 452), (186, 284)]]
[[(250, 453), (190, 283)]]
[[(250, 451), (200, 278)]]
[[(251, 452), (201, 277)]]
[[(250, 451), (210, 276)]]
[

In [10]:
#run these 2 if ur cam crash (copy and run somewhere else)
cap.release()
cv2.destroyAllWindows()  